In [2]:
import os
import pandas as pd
import rasterio

In [3]:
def is_x_constant(raster):
    return raster.xy(0, 0)[0] == raster.xy(raster.height-1, raster.width-1)[0]

def is_y_constant(raster):
    return raster.xy(0, 0)[1] == raster.xy(raster.height-1, raster.width-1)[1]

files = os.listdir('../data/city_rasters/')
cities = ['aleppo', 'damascus', 'raqqa', 'daraa', 'homs', 'idlib', 'hama', 'deir']
good_files, bad_files = [], []
for city in cities:
    city_files = [file for file in files if city in file and file.endswith('bmp')]
    for file in city_files:
        raster = rasterio.open(f'../data/city_rasters/{file}')
        x_is_constant, y_is_constant = is_x_constant(raster), is_y_constant(raster)
        
        if x_is_constant or y_is_constant:
            bad_files.append((file, x_is_constant, y_is_constant))
            
        if not x_is_constant and not y_is_constant:
            good_files.append(file)

/Users/yc00031/.virtualenvs/py3/lib/python3.7/site-packages/rasterio/__init__.py:216: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, **kwargs)


On these files, either the latitude or the longitude (or both) are equal on the boundaries of the image (we assume it's contant across the entire image). Hence, we cannot use them for linking them with the annotation.

In [4]:
bad_files

[]

On these files, that problem is not present

In [5]:
good_files

['daraa_2016_02_25_zoom_19.bmp']

If images have the same dimensions, we could try to infer the coordinates from those images that work.

In [7]:
files = os.listdir('../data/city_rasters/')
cities = ['aleppo', 'damascus', 'raqqa', 'daraa', 'homs', 'idlib', 'hama', 'deir']
dimensions = []
for city in cities:
    city_files = [file for file in files if city in file]
    for file in city_files:
        raster = rasterio.open(f'../data/city_rasters/{file}')
        dimensions.append({
            'city': city,
            'width': raster.width,
            'height': raster.height,
            'x_is_constant': is_x_constant(raster),
            'y_is_constant': is_y_constant(raster)
        })

In [11]:
pd.DataFrame(dimensions).groupby('city')['height'].unique()

city
aleppo      [25088]
damascus     [8192]
daraa       [12032]
deir        [11264]
hama        [15616]
homs        [18176]
idlib        [8704]
raqqa        [8960]
Name: height, dtype: object

In [12]:
pd.DataFrame(dimensions).groupby('city')['width'].unique()

city
aleppo      [37376]
damascus    [11776]
daraa       [13312]
deir        [12032]
hama        [15616]
homs        [20736]
idlib       [10752]
raqqa       [23552]
Name: width, dtype: object

In [5]:
raster = rasterio.open(f'../data/test/homs_2016_05_30_v3/homs_2016_05_30_v3_zoom_19.tif')
x_is_constant, y_is_constant = is_x_constant(raster), is_y_constant(raster)

if x_is_constant or y_is_constant:
    bad_files.append((file, x_is_constant, y_is_constant))

if not x_is_constant and not y_is_constant:
    good_files.append(file)

In [6]:
x_is_constant

True

In [8]:
raster.xy(10000, 10000)

(36.0, 34.0)